**Optimal Power Flow Seminar**

**Power Markets and Regulations**

**TA: Sahar Moghimian**

<!--NOTEBOOK_HEADER-->
*Most of the materials used for this notebook and our future seminars were developed by Prof. Michael Davidson and Prof. Jesse Jenkins and are used at the following courses:*

- *MAE / ENE 539 Optimization Methods for Energy Systems Engineering (Advanced Topics in Combustion I) [Princeton]*

- *MAE 243 Electric Power Systems Modeling [UC San Diego]*

*The full Github Repo with materials can be found* [here](https://github.com/Power-Systems-Optimization-Course/power-systems-optimization?tab=readme-ov-file#readme)

---
*This notebook has been adapted to Python/Pyomo from the original Julia/JuMP version.*

#**Optimal Power Flow**
<img src = "https://raw.githubusercontent.com/Saharmgh/OPFtest/main/intro%20(1).png" style="width: 10px; height: 10px" align = "left">  


## **Power Systems Optimization**

This notebook provides an introduction to the Optimal Power Flow (OPF) problem in electric power systems&mdash;which minimizes the short-run production costs of meeting electricity demand at a number of connected locations from a given set of generators subject to various technical and transmission network flow limit constraints. This will be our first treatment of a *network*, which is critical to all power systems.

We will introduce a linear approximation to the AC optimal power flow problem known as "**DC-OPF**", where we begin to incorporate some of the physics involved in how electricity flows along transmission lines. With this formulation, we recognize that given "injections" (i.e., generation) and "withdrawals" (i.e., demand) of power at each node in the network, flows along lines are not independently controllable. Instead, electrical power flows across transmission lines in relation to their physical properties, namely power flows across parallel circuits or paths in inverse proportion to the [electrical impedance](https://en.wikipedia.org/wiki/Electrical_impedance) of the lines. This can (very frequently) result in hitting flow constraints before we would if could control power flows across all lines as in the transport problem.

## Introduction to OPF


The Optimal Power Flow (OPF) problem is a power system optimal scheduling problem which captures the physics of electricity flows across electricity networks, adding a layer of complexity and more realism to the Economic Dispatch (ED) problem. OPF usually attempts to capture the entire network topology by representing the transmission line interconnections between different nodes (also known as buses, or locations where generators or demand inject or withdraw power into/from the network) including various electrical parameters, such as resistance, series reactance, shunt admittance, etc. The full alternating current or "AC" OPF is a non-convex problem and turns out to be an extremely hard problem to solve (usually NP-hard). Hence, system operators and power marketers usually go about solving a linearized version of it, called the "DC-OPF." The DC-OPF approximation works satisfactorily for bulk power transmission networks as long as such networks are not operated at the brink of instability or under heavily loaded conditions.

### 1. Install and load packages

In [ ]:
# Install required packages (run this cell once)
!pip install -q pyomo highspy pandas numpy

In [ ]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 120)
pd.set_option('display.width', 120)

### 2. Load and format data

We will load a modified 3-bus case stored in the [MATPOWER case format](https://matpower.org/docs/ref/matpower5.0/caseformat.html). It consists of:

- two generator buses where 1000 MW generators are located, one with variable cost of 50/MWh and another with variable cost of 100/MWh
- one load bus where 600 MW of demand is located
- three lines connecting the buses, each with a maximum flow of 500 MW

The location and numbering of the components:

<img src="https://github.com/Power-Systems-Optimization-Course/power-systems-optimization/blob/master/Notebooks/img/opf_network.png?raw=1" style="width: 450px; height: auto" align="left">

In [ ]:
# Reading the data from GitHub
base_url = "https://raw.githubusercontent.com/Power-Systems-Optimization-Course/power-systems-optimization/master/Notebooks/opf_data/"

gen = pd.read_csv(base_url + "gen.csv")
gencost = pd.read_csv(base_url + "gencost.csv")
branch_df = pd.read_csv(base_url + "branch.csv")
bus = pd.read_csv(base_url + "bus.csv")

In [ ]:
# Rename all columns to lowercase (by convention)
for df in [gen, gencost, branch_df, bus]:
    df.columns = df.columns.str.lower()

Let's make the data more organized;

In [ ]:
# Create generator ids
gen['id'] = range(1, len(gen) + 1)
gencost['id'] = range(1, len(gencost) + 1)

Our data files for the test system contain parameters for resistance and reactance of the transmission lines, which are related to complex impedance:

$$
Z = R + iX
$$

where $R$ = resistance is the real part, $X$ = reactance is the imaginary part. Recall from above that impedance is the inverse of admittance; hence, we have the following transformation for susceptance:

$$
B = \text{Im}\left(\frac{1}{R + iX}\right) = \frac{-X}{|R + iX|^2} = \frac{-X}{R^2 + X^2}
$$

But, since we neglect the resistance for the purpose of solving the DC-OPF, we can approximate the susceptance from above as:

$$
B = \frac{1}{X}
$$

The data are converted to have positive values of both $X$ and $B$, hence we remove the negative sign. See [link](https://en.wikipedia.org/wiki/Susceptance#Relationship_to_reactance)

In [ ]:
# Create line ids
branch_df['id'] = range(1, len(branch_df) + 1)

# Create other lines with reverse flow direction (bij --> bji)
branch1 = branch_df.copy()
branch2 = branch1.copy()
branch2['fbus'], branch2['tbus'] = branch1['tbus'].values, branch1['fbus'].values
branch2 = branch2[branch1.columns]
branch = pd.concat([branch1, branch2], ignore_index=True)

# Calculate the susceptance of each line, on the assumption that
# reactance is >> resistance.
branch['sus'] = 1.0 / branch['x']

In [ ]:
bus

Columns pd and qd indicate the [active and reactive power](https://en.wikipedia.org/wiki/AC_power#Active,_reactive,_and_apparent_power) withdrawal at the bus. (We will ignore qd for this notebook, since we are not considering full AC power flows.) We do not need any of other columns for our purposes.

In [ ]:
gen

In [ ]:
# And the generator cost dataset:
gencost

In the above, model=2 indicates a polynomial variable cost formulation and the column n=2 indicates that there are two terms. Thus, we have a linear cost (in the x1 column) without any quadratic terms (and a zero constant term):

$$
VarCost_g = x1_g
$$

In [ ]:
# Here are the transmission lines:
branch

<!-- Let's try to understand Branch dataset:

<img src = "https://raw.githubusercontent.com/Saharmgh/OPFtest/main/ij13.png" style="width: 5px; height: 5px" align = "left"> Note that while there are three lines, there are six entries here. There is an entry for each 'direction' of flow the lines can accomodate, hence six entries for three lines. The column `fbus` denotes the ID of the "from bus" or origin bus and the column `tbus` denotes the ID of the "to bus" or destination bus (e.g. `fbus`=1, `tbus`=3 is the flow in the direction from bus 1 to bus 3). -->


For this model formulation, we are only using transmission line capacity (known as "ratings"), given in `ratea`, `rateb`, and `ratec`. These correspond to different ratings based on how long the line might be overloaded, with `ratec` known as an "emergency rating", which could exceed the long-term rating, `ratea`. We will use `ratea` for this model. The dataset also contains resistance and reactance.

## The "DC" Optimal Power Flow problem

The previosuly mentioned models are not physically correct as we cannot arbitrarily route power through lines. We will now introduce a linear approximation to the optimal power flow problem that incorporates this limitation and is tractable and reasonably accurate. This is commonly called the "DC" optimal power flow problem, but in reality (as we'll see below), it is a linearization that still relates to the physics of AC power flows, it just simplifies (and ignores) certain non-convexities to produce a tractable linear programming problem that remains a valid approximation under certain circumstances.

In the "DC" or linear approximation of the AC optimal power flow problem, power flows along a line from bus $i$ to bus $j$ are driven by voltage [phase angle](https://en.wikipedia.org/wiki/Phasor) differences, denoted by $\theta_i$ and $\theta_j$:

$$
FLOW_{ij} = BaseMVA \times B_{ij} (\theta_i-\theta_j)
$$

Where $FLOW_{ij}$ is the flow across the line from node $i$ to node $j$ (in MW), $BaseMVA$ is the base power for the network (in MVA), $B_{ij}$ is the [susceptance](https://en.wikipedia.org/wiki/Susceptance) for the line connecting buses $i$ and $j$ (in per unit terms) and $(\theta_i-\theta_j)$ is the difference in voltage angles between buses (in radians).

Susceptance is the imaginary part of the [admittance](https://en.wikipedia.org/wiki/Admittance) of a line, where [admittance](https://en.wikipedia.org/wiki/Admittance) is a complex number that describes how easy it is for AC current to flow across a given conductor. Power flows in parallel circuits in an AC network in proportion to their admittance (or inverse proportion to [impedance](https://en.wikipedia.org/wiki/Electrical_impedance), which describes how hard it is a measure of the opposition that a circuit presents to a current when a voltage is applied).

Voltage [phase](https://www.allaboutcircuits.com/textbook/alternating-current/chpt-1/ac-phase/) angles describe the displacement of the AC voltage waveform at each node, relative to a reference or "slack" bus. A difference in voltage angles between buses $i$ and $j$ indicates that the peaks and troughs in the sinusoidal voltage waveform at bus $i$ are shifted in time relative to the voltage waveform at bus $j$, as in the image below.

<img src="https://github.com/Power-Systems-Optimization-Course/power-systems-optimization/blob/master/Notebooks/img/phase_shift.png?raw=1" style="width: 450px; height: auto"> (*Image source: [allaboutcircuits.com](https://www.allaboutcircuits.com/textbook/alternating-current/chpt-1/ac-phase/)*)

In AC circuits, power flows from nodes with higher voltage angle to buses with lower voltage angle, just as power flows from higher voltage magnitude to lower voltage magnitude in DC circuits.

For further reading:



What causes the shift in voltage phase angle? An AC current flowing across a conductor encountering either [inductive reactance](https://en.wikipedia.org/wiki/Electrical_impedance#Inductive_reactance) (which relates to the magnetizing current, or the energy required to continually induce or establish magnetic fields around a conductor as AC current polarity flips each cycle) or [capacitive reactance](https://en.wikipedia.org/wiki/Electrical_impedance#Capacitive_reactance) (which relates to the charging current, or the energy required to sustain capacitive charges between two conductors separated by an insulator) will experience a shift in the voltage waveform, relative to the current wave form. Inductive reactance causes the voltage waveform to shift forward relative to the current, while capacitive reactive cases the voltage to shift backwards relative to the current. In overhead transmission lines, the primary source of voltage phase angle shifts is the inductance of the transmission lines themselves (although capacitance is relevant for underground lines).

(For (a bit) more on the physics of power flow on AC transmissio lines, you can review [this tutorial from the PJM Interconnection](https://learn.pjm.com/~/media/training/nerc-certifications/gen-exam-materials/bet/20160104-basics-of-elec-power-flow-on-ac.ashx))

The reason this approximation is known as the "DC OPF" is because we ignore [reactive power](https://en.wikipedia.org/wiki/AC_power#Reactive_power_flows) and focus only on flows of real power as in a DC network. But you can see why "DC" OPF is actually a misnomer: we're still dealing with AC voltages and susceptance terms here, we're just linearizing the problem through some simplifying assumptions.

###Assumptions
 In particular, the three basic assumptions used to derive a linearized or "DC" OPF approximation from the underlying AC OPF problem are as follows:

1. The resistance for each branch is negligible relative to the reactance, and can therefore be approximated as ~0.
2. The voltage magnitude at each bus is constant and equal to the base voltage (e.g. equal to 1 p.u).
3. The voltage angle difference $(\theta_j-\theta_i)$ across any branch from bus $i$ to $j$ is sufficiently small such that $cos(\theta_i-\theta_j) \approx 1$ and $sin(\theta_i-\theta_j) \approx (\theta_i-\theta_j)$. Note that $\theta_i,\theta_j$ are measured in [radians](https://en.wikipedia.org/wiki/Radian).

Fortunately, under normal operating conditions, these conditions hold for electricity transmission networks (although note they are not generally acceptable simplifying assumptions for lower voltage distribution networks).

Here there is the DCOPF-related decision variables and constraints:

$$
\begin{align}
\min \ & \sum_{g \in G} VarCost_g \times GEN_g & \\
\text{s.t.} & \\
 & \sum_{g \in G_i} GEN_g - Demand_i = \sum_{j\in J(i)} FLOW_{i,j} & \forall \quad i \in \mathcal{N}\\
 & FLOW_{i,j} \leq MaxFlow_{ij} & \forall \quad i \in \mathcal{N}, \forall j \in J_i \\
 & FLOW_{i,j} = BaseMVA \times B_{ij}(\theta_i-\theta_j) & \forall \quad i \in \mathcal{N}, \forall j \in J_i \\
 & GEN_g \leq Pmax_g & \forall \quad g \in G \\
 & GEN_g \geq Pmin_g & \forall \quad g \in G \\
 & \theta_{slack} = 0
\end{align}
$$

<!-- Note that we no longer require a constraint to enforce anti-symmetric flows. -->

We have the following **sets**:
- $\mathcal{N}$, the set of all nodes (or buses) in the network
- $J_i \subset \mathcal{N}$, the subset of nodes that are connected to node $i$
- $G_i \subset G$, the subset of generators located at node $i$

The **decision variables** in the above problem are:

- $GEN_{g}$, the generation (in MW) produced by each generator, $g$
- $\theta_i$, the voltage phase angle at bus $i$ relative to the slack or reference bus ($\theta_{slack}$)
- $FLOW_{i,j}$, the flow from bus i to bus j (in MW)

In the OPF problem we *do not* directly choose the flows across lines, but rather choose the real power injections at generator buses via $GEN_{g}$ and the voltage angles $\theta_i$, which collectively determine the power flows across lines via the collection of constraints above. The $FLOW$ decision variable is thus an "auxialiary" variable, as it is precisely determined by the constraint $FLOW_{i,j} = BaseMVA \times B_{ij}(\theta_i-\theta_j)$ for all pairs ($i,j$) for which transmission lines exist.

(Note that we create $FLOW$ decisions for some pairs of nodes ($i,j$) that are not connected by lines; these variables are "free" variables, as they are not constrained and do not show up in/effect the objective function, so the solver will generally remove these variables in pre-solve step. We could try a different approach to sets for the $FLOW$ variable, e.g. by setting across lines instead of pairs of nodes, to avoid creating these unecessary free variables, but for convenience, we'll create and ignore them for now).

The **parameters** are:

- $Pmin_g$, the minimum operating bounds for the generator (based on engineering or natural resource constraints)
- $Pmax_g$, the maximum operating bounds for the generator (based on engineering or natural resource constraints)
- $Demand_i$, the demand (in MW) at bus $i$
- $MaxFlow_{ij}$, the maximum allowable flow along the line from $i$ to $j$
- $VarCost_g$, the variable cost of generator $g$
- $B_{ij}$, susceptance for line connecting buses $i$ and $j$
- $\theta_{slack}$, the "slack" bus or reference bus from which relative voltage angles at all other buses are calcluated. Thus $\theta_{slack}=0$.
- $BaseMVA$, the base power in MVA for the network (used to scale from standard units to per unit values or vice versa).

### 3. Create solver function (dcopf)

We have a linear problem so we simply use [HiGHS](https://en.wikipedia.org/wiki/HiGHS_optimization_solver) optimizer, it is an open-source solver which is wrapped by Pyomo to solve linear programming (LP), mixed-integer programming (MIP), and convex quadratic programming (QP) models. You are free to choose other available [solvers](https://www.pyomo.org/installation).

In [ ]:
def dcopf(gen, branch, gencost, bus):
    """
    Solve DC Optimal Power Flow problem (3-bus system).
    
    Parameters:
        gen      -- DataFrame with generator info
        branch   -- DataFrame with transmission line info (including reverse directions)
        gencost  -- DataFrame with generator cost info
        bus      -- DataFrame with bus types and loads
    """
    model = pyo.ConcreteModel()
    
    # Enable dual extraction for LMPs
    model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    
    # Define sets
    # Set of all generators
    G = list(gen['id'])
    # Set of all nodes
    N = list(bus['bus_i'])
    # sets J_i and G_i will be described using dataframe indexing below
    
    # Define per unit base units for the system
    # used to convert from per unit values to standard unit
    # values (e.g. p.u. power flows to MW/MVA)
    baseMVA = gen['mbase'].iloc[0]  # base MVA is 100 MVA for this system
    
    # Flow pairs from the branch dataframe (includes both directions)
    flow_pairs = list(zip(branch['fbus'], branch['tbus']))
    
    # Decision variables
    model.GEN = pyo.Var(G, within=pyo.NonNegativeReals)
    # Note: we assume Pmin = 0 for all resources for simplicity here
    model.THETA = pyo.Var(N, within=pyo.Reals)  # voltage phase angle of bus
    model.FLOW = pyo.Var(flow_pairs, within=pyo.Reals)  # flows between connected pairs of nodes
    
    # Boundary condition
    # Create slack bus with reference angle = 0
    # Note: by convention this is a generator bus. Hence, we will select bus 1
    model.THETA[1].fix(0)
    
    # Objective function
    model.obj = pyo.Objective(
        expr=sum(
            gencost.loc[gencost['id'] == g, 'x1'].iloc[0] * model.GEN[g]
            for g in G
        ),
        sense=pyo.minimize
    )
    
    # Nodal balance constraints
    # LMPs come from the duals of these constraints
    model.cBalance = pyo.Constraint(N)
    for i in N:
        # Generators at bus i
        gens_at_i = list(gen.loc[gen['bus'] == i, 'id'])
        # Demand at bus i
        demand_i = bus.loc[bus['bus_i'] == i, 'pd'].iloc[0]
        # Lines leaving bus i
        lines_from_i = branch.loc[branch['fbus'] == i]
        tbus_list = list(lines_from_i['tbus'])
        
        model.cBalance[i] = (
            sum(model.GEN[g] for g in gens_at_i) - demand_i
            == sum(model.FLOW[i, j] for j in tbus_list)
        )
    
    # Technical constraints
    # Max generation constraint
    model.cMaxGen = pyo.Constraint(G)
    for g in G:
        model.cMaxGen[g] = model.GEN[g] <= gen.loc[gen['id'] == g, 'pmax'].iloc[0]
    
    # Max line flow constraints
    model.cLineLimits = pyo.ConstraintList()
    for l in range(len(branch)):
        fbus = branch.iloc[l]['fbus']
        tbus = branch.iloc[l]['tbus']
        model.cLineLimits.add(
            model.FLOW[fbus, tbus] <= branch.iloc[l]['ratea']
        )
    
    # Power Flow Equations
    # In DCOPF, line flow is a function of voltage angles
    model.cLineFlows = pyo.ConstraintList()
    for l in range(len(branch)):
        fbus = branch.iloc[l]['fbus']
        tbus = branch.iloc[l]['tbus']
        sus = branch.iloc[l]['sus']
        model.cLineFlows.add(
            model.FLOW[fbus, tbus]
            == baseMVA * sus * (model.THETA[fbus] - model.THETA[tbus])
        )
    
    # Solve
    solver = pyo.SolverFactory('appsi_highs')
    result = solver.solve(model, tee=True)
    
    # Output variables
    generation = pd.DataFrame({
        'id': gen['id'].values,
        'node': gen['bus'].values,
        'gen': [pyo.value(model.GEN[g]) for g in G]
    })
    
    angles = {i: pyo.value(model.THETA[i]) for i in N}
    
    flows = pd.DataFrame({
        'fbus': branch['fbus'].values,
        'tbus': branch['tbus'].values,
        'flow': [
            baseMVA * branch.iloc[l]['sus'] * (angles[branch.iloc[l]['fbus']] - angles[branch.iloc[l]['tbus']])
            for l in range(len(branch))
        ]
    })
    
    # We output the marginal values of the demand constraints,
    # which will in fact be the prices to deliver power at a given bus.
    prices = pd.DataFrame({
        'node': N,
        'value': [model.dual[model.cBalance[i]] for i in N]
    })
    
    # Return the solution and objective as a dictionary
    return {
        'generation': generation,
        'angles': angles,
        'flows': flows,
        'prices': prices,
        'cost': pyo.value(model.obj),
        'status': str(result.solver.termination_condition)
    }

### 4. Solve

In [ ]:
solution = dcopf(gen, branch, gencost, bus)

In [ ]:
solution['generation']

Hence, we still generate all 600 MW from Gen A at Bus 1.

In [ ]:
# These are the voltage phase angles of the buses relative to Bus 1.
angles_df = pd.DataFrame({
    'Theta1': [solution['angles'][1]],
    'Theta2': [solution['angles'][2]],
    'Theta3': [solution['angles'][3]]
})
angles_df

The following flows are created:

- $l_{1,3}$ = 400 MW
- $l_{1,2}$ = 200 MW
- $l_{2,3}$ = 200 MW

<!-- The reason we can't make maximum use of line $l_{1,3}$ is because power flows split across parallel circuits in inverse proportion to the impedance of the circuit paths. Here, all three branches have equal susceptance, and thus equal impedance (since we are assuming resistance is ~0). Thus, the path $l_{1,2} \rightarrow l_{2,3}$ has twice the impedance as the path $l_{1,3}$ and thus takes half as much power flow coming from Gen A at Bus 1. -->

In [ ]:
solution['flows']

In [ ]:
print(f"Total cost: ${solution['cost']:,.2f}")
print(f"Status: {solution['status']}")

### 5. Solve high demand case

Now, let's increase demand at Bus 3 to 800 MW. Despite spare capacity at Gen A, it turns out we will no longer be able to generate all of our power from Gen A alone.

In [ ]:
bus_high = bus.copy()
bus_high.loc[bus_high['bus_i'] == 3, 'pd'] = 800  # set demand at bus 3 to 800 MW

sol_high = dcopf(gen, branch, gencost, bus_high)
sol_high['generation']

This situation is explained by flow patterns, where the capacity of $l_{13}$ is at its maximum, but in order to meet demand at Bus 3, more power needs to be injected in Bus 2, requiring the more costly generator at Bus 2 to dispatch, despite spare capacity at the generator at Bus 1.

In [ ]:
sol_high['flows']

In [ ]:
# Voltage phase angles for the high demand case
angles_high_df = pd.DataFrame({
    'Theta1': [sol_high['angles'][1]],
    'Theta2': [sol_high['angles'][2]],
    'Theta3': [sol_high['angles'][3]]
})
angles_high_df

The following flows are created:

- $l_{1,3}$ = 500 MW
- $l_{1,2}$ = 200 MW
- $l_{2,3}$ = 300 MW

What is going on here?

Generator 1 at Bus 1 produces 700 MW, which must split in proportion to impedance once again, with 2/3 of the power or 466.67 MW flowing along $l_{13}$ and 233.33 MW must flow along the route $l_{1,2} \rightarrow l_{2,3}$ with double the impedance.

At the same time, the 100 MW injected by generator 2 at Bus 2 *also* splits with 2/3 or 66.67 flowing along $l_{2,3}$ and 1/3 or 33.33 along $l_{2,1} \rightarrow l_{1,3}$.

The total flows across each segment are thus:

- $l_{1,3}$ = 466.67 + 33.33 = 500 MW
- $l_{1,2}$ = 233.33 - 33.33 = 200 MW
- $l_{2,3}$ = 233. 33 + 66.67 = 300 MW

### 6. Compare prices

The marginal values of the demand constraints at a given bus represent the change in the objective that results from increasing demand at the bus by one unit. This is the natural definition of a "value" of power at that location, and is the basis for **[locational marginal prices](https://www.iso-ne.com/participate/support/faq/lmp)** (LMPs) found in electricity markets.

We examine first the regular case of demand = 600 MW, then the high demand case = 800 MW.

In [ ]:
solution['prices']

All prices are the same in this case. The interpretation: if we were to add an incremental load at any of the buses, we could meet it from additional production from Gen A which has marginal cost of \$50 / MWh. We are not going to hit any transmission limits.

In [ ]:
sol_high['prices']

Something interesting has happened!

First, note that the prices are different. Hence, we will not be able to meet incremental load from production by Gen A (except if we add load right at Gen A located at Bus 1). Similarly, load at Bus 2 can be met by increasing production from Gen B with marginal cost = \$100 / MWh.

However, why does Bus 3 have a marginal price of \$150 / MWh?! That's higher than the marginal cost of either of our two generators?

The answer lies in what must happen to meet an incremental load at Bus 3 while respecting transmission constraints. We must increase from Gen B, but in doing so, part of the power from Gen B will go through $l_{2,1} \rightarrow l_{1,3}$ in addition to $l_{2,3}$, since power flows split across parallel paths in proportion to admittance (or inverse proportion to impedance). However, without adjusting Gen A's output, an increase in production from Gen B will cause us to exceed the transmission constraint on line $l_{1,3}$, requiring us to throttle back power from Gen A to keep power flows feasible.

The exact change in generation for an incremental 1 MW load at Bus 3 is thus:
- Gen B $\uparrow$ 2 MW
- Gen A $\downarrow$ 1 MW

Hence:

$$
Price_3 = 2 \times VarCost_B - VarCost_A = \$150 \text{ / MWh}
$$

In a network with thousands of nodes and many parallel paths and loop flows, one can see quite quickly how prices may vary in unexpected ways; hence, the need for detailed mathematical models to compute locational marginal prices.

### 7. The IEEE 14 bus test system

We now explore a complicated system, the 14-bus IEEE test system, illustrated here:
The IEEE 14-bus test case represents a simple approximation of the American Electric Power system as of February 1962 [1]. It has 14 buses, 5 generators, and 11 loads


<img src="https://github.com/Power-Systems-Optimization-Course/power-systems-optimization/blob/master/Notebooks/ieee_test_cases/IEEE14BusTestSystem.png?raw=1" style="width: 450px; height: auto" align="center">

The system consists of:
- 2 generators (located at nodes 1 and 2)
- 11 loads
- a meshed transmission network including transformers and multiple voltages


In [ ]:
ieee_url = "https://raw.githubusercontent.com/Power-Systems-Optimization-Course/power-systems-optimization/master/Notebooks/ieee_test_cases/"

gens = pd.read_csv(ieee_url + "Gen14.csv")
loads = pd.read_csv(ieee_url + "Load14.csv")
lines_df = pd.read_csv(ieee_url + "Tran14_b.csv")

In [ ]:
# Rename all columns to lowercase
for df in [gens, lines_df, loads]:
    df.columns = df.columns.str.lower()

In [ ]:
# Create generator ids
gens['id'] = range(1, len(gens) + 1)

In [ ]:
# Create line ids
lines_df['id'] = range(1, len(lines_df) + 1)

In [ ]:
# Add set of rows for reverse direction with same parameters
lines1 = lines_df.copy()
lines2 = lines1.copy()
lines2['fromnode'], lines2['tonode'] = lines1['tonode'].values, lines1['fromnode'].values
lines2 = lines2[lines1.columns]
lines = pd.concat([lines1, lines2], ignore_index=True)

# Calculate simple susceptance, ignoring resistance as earlier
lines['b'] = 1.0 / lines['reactance']

In [ ]:
# Keep only a single time period
loads = loads[['connnode', 'interval-1_load']].copy()
loads.rename(columns={'interval-1_load': 'demand'}, inplace=True)

In [ ]:
lines

In [ ]:
gens

In [ ]:
loads

The structure of these data are different than the above case formats, hence we write a modified solver function. Try to follow the comments:

#### Exercise: Complete the DC-OPF for the IEEE 14-bus system

The function below has the decision variables defined but the constraints are left as comments for you to fill in. Try completing it yourself before looking at the solution in the next cell.

In [ ]:
def dcopf_ieee_exercise(gens, lines, loads):
    """
    EXERCISE: Solve DC OPF problem using IEEE test cases.
    Fill in the constraints below.
    
    Inputs:
        gens  -- DataFrame with generator info
        lines -- DataFrame with transmission line info
        loads -- DataFrame with load info
    """
    model = pyo.ConcreteModel()
    model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    
    # Define sets based on data
    # Set of generator buses
    G = list(gens['connnode'])
    
    # Set of all nodes
    N = sorted(set(lines['fromnode']).union(set(lines['tonode'])))
    
    baseMVA = 100  # base MVA is 100 MVA for this system
    
    # Flow pairs from the lines dataframe
    flow_pairs = list(zip(lines['fromnode'], lines['tonode']))
    
    #*******************************************************************************
    # Decision variables
    model.GEN = pyo.Var(N, within=pyo.NonNegativeReals)  # generation at each node
    model.THETA = pyo.Var(N, within=pyo.Reals)            # voltage phase angle
    model.FLOW = pyo.Var(flow_pairs, within=pyo.Reals)    # flows between connected nodes
    
    # Create slack bus with reference angle = 0; use bus 1 with generator
    # fix ...
    
    # Objective function
    # model.obj = pyo.Objective(...)
    
    # Nodal balance equations
    # model.cBalance = pyo.Constraint(N)
    # for i in N:
    #     ...
    
    # Max generation constraint
    # model.cMaxGen = ...
    
    # Max line flow constraints
    # model.cLineLimits = ...
    
    # Power flow equations
    # model.cLineFlows = ...
    
    #*******************************************************************************
    
    # Solve
    solver = pyo.SolverFactory('appsi_highs')
    result = solver.solve(model, tee=True)
    
    # Output variables
    generation = pd.DataFrame({
        'node': gens['connnode'].values,
        'gen': [pyo.value(model.GEN[g]) for g in G]
    })
    
    angles = {i: pyo.value(model.THETA[i]) for i in N}
    
    flows = pd.DataFrame({
        'fbus': lines['fromnode'].values,
        'tbus': lines['tonode'].values,
        'flow': [
            baseMVA * lines.iloc[l]['b'] * (angles[lines.iloc[l]['fromnode']] - angles[lines.iloc[l]['tonode']])
            for l in range(len(lines))
        ]
    })
    
    prices = pd.DataFrame({
        'node': N,
        'value': [model.dual[model.cBalance[i]] for i in N]
    })
    
    return {
        'generation': generation,
        'angles': angles,
        'flows': flows,
        'prices': prices,
        'cost': pyo.value(model.obj),
        'status': str(result.solver.termination_condition)
    }

#### Solution: Complete DC-OPF for the IEEE 14-bus system

In [ ]:
# Your code here
# Complete the dcopf_ieee() function based on the exercise template above
# Then solve and examine the results

def dcopf_ieee(gens, lines, loads):
    """
    Solve DC OPF problem using IEEE test cases.
    Complete this function!
    """
    # Your implementation here
    pass

In [ ]:
solution_ieee = dcopf_ieee(gens, lines, loads)

In [ ]:
solution_ieee['generation']

In [ ]:
# Voltage phase angles
angles_ieee = pd.DataFrame({
    'node': list(solution_ieee['angles'].keys()),
    'theta': list(solution_ieee['angles'].values())
})
angles_ieee

In [ ]:
solution_ieee['prices']

Note that this system is uncongested, we ignore transmissions losses here, and the lowest cost generator supplies demand at all nodes, leading to equal locational marginal price at all nodes. Later you can explore more complicated examples and modify this test system...

In [ ]:
# Line flows for IEEE 14-bus system
# Show only the original direction flows (first half of the dataframe)
n_original = len(lines_df)
solution_ieee['flows'].head(n_original)

In [ ]:
print(f"Total cost: ${solution_ieee['cost']:,.2f}")
print(f"Status: {solution_ieee['status']}")
print(f"\nAll LMPs are approximately equal at ${solution_ieee['prices']['value'].mean():.2f}/MWh")
print("This confirms the system is uncongested -- no binding transmission constraints.")